En esta versión, el modelo Ridge Regression se utiliza como modelo base (baseline) para predecir el valor máximo de FRP (frp_max) y, a partir de dicha predicción continua, construir una clasificación operativa de incendios intensos.

**Criterio operativo**

Se define un evento intenso de la siguiente manera:

Evento intenso = 1 ⇔ FRP máximo predicho ≥ 100

Este criterio transforma el problema original de regresión en una clasificación derivada, orientada a la toma de decisiones operativas.

**Pipeline del modelo**

El flujo de trabajo del modelo se estructura en las siguientes etapas:

Carga del dataset consolidado, que contiene variables climáticas, métricas de FRP y la etiqueta real de riesgo.

Entrenamiento del modelo Ridge Regression, cuyo objetivo es predecir el valor máximo de FRP (frp_max) a partir de las variables predictoras.

Generación de predicciones continuas, obteniendo la variable
frp_max_pred_ridge.

Clasificación operativa derivada, donde se asigna:

evento_intenso_pred = 1 si frp_max_pred_ridge ≥ 100,

evento_intenso_pred = 0 en caso contrario.

Construcción de un dataset de salida, incorporando las nuevas columnas de predicción y clasificación, manteniendo la trazabilidad con los eventos originales.

Evaluación del desempeño en clasificación, utilizando:

métricas estándar (precision, recall y F1-score),

matriz de confusión visualizada con paleta Blues, con los valores de TN, FP, FN y TP explícitamente rotulados.

*Al evaluar la clasificación derivada para un umbral de FRP ≥ 100 MW, se observa que el modelo Ridge presenta una sensibilidad extremadamente baja (recall ≈ 0.04), a pesar de una precisión moderada. Este comportamiento se explica por la alta regularización del modelo y por el hecho de que el error medio de predicción es del mismo orden de magnitud que el umbral considerado. En consecuencia, el modelo tiende a subestimar eventos intensos, confirmando que enfoques lineales no son adecuados para la detección de incendios de intensidad moderada.*

In [ ]:
# ============================================================
# MODELO 1 — RIDGE REGRESSION
# ============================================================
# Descripción general:
# Este script implementa un modelo de regresión lineal regularizada (Ridge)
# para predecir el valor máximo de FRP (frp_max) de eventos de incendio.
# A partir de esta predicción continua, se construye una clasificación
# operativa de incendios intensos utilizando un umbral fijo (FRP ≥ 100).
#
# El modelo Ridge se utiliza como baseline lineal para comparar su desempeño
# con modelos no lineales más complejos (Random Forest y MLP).
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    f1_score
)


# -----------------------------
# 0) FUNCIÓN MATRIZ DE CONFUSIÓN (PALETA BLUES)
# -----------------------------
# Descripción:
# Se define una función auxiliar para visualizar la matriz de confusión
# correspondiente a la clasificación derivada. La visualización muestra
# explícitamente los verdaderos y falsos positivos y negativos, facilitando
# la interpretación del desempeño del modelo en términos operativos.

def plot_confusion_with_labels_blues(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()

    labels = np.array(
        [
            [f"TN\n{tn}", f"FP\n{fp}"],
            [f"FN\n{fn}", f"TP\n{tp}"]
        ]
    )

    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(cm, cmap="Blues")

    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(["Pred 0", "Pred 1"])
    ax.set_yticklabels(["Real 0", "Real 1"])
    ax.set_xlabel("Predicción")
    ax.set_ylabel("Real")
    ax.set_title(title)

    threshold = cm.max() / 2 if cm.max() > 0 else 0
    for i in range(2):
        for j in range(2):
            color = "white" if cm[i, j] > threshold else "black"
            ax.text(j, i, labels[i, j], ha="center", va="center", color=color)

    plt.colorbar(im, ax=ax)
    plt.grid(False)
    plt.tight_layout()
    plt.show()


# -----------------------------
# 1) CARGA DEL DATASET
# -----------------------------
# Descripción:
# Se carga el dataset consolidado de eventos de incendio, que contiene variables
# climáticas, métricas de FRP y una etiqueta binaria real de riesgo. Este dataset
# constituye la base para el entrenamiento y evaluación del modelo Ridge.

DATA_PATH = "/content/sample_data/Dataset_Incendios_Eventos_TARGET_RIESGO_FRP.csv"
df = pd.read_csv(DATA_PATH)

print("✅ Dataset cargado")
print("Shape:", df.shape)
display(df.head())
print("Columnas disponibles:", df.columns.tolist())


# -----------------------------
# 2) CONFIGURACIÓN DE COLUMNAS
# -----------------------------
# Descripción:
# Se definen explícitamente la variable objetivo de regresión, la etiqueta
# real de clasificación y el conjunto de variables predictoras. Esta etapa
# asegura coherencia entre el dataset y el diseño del modelo.

TARGET_REG = "frp_max"
TARGET_CLASS_REAL = "target_riesgo"
FRP_INICIAL_COL = "frp_inicial"

features = [
    "frp_inicial",
    "temperature_2m_mean", "relativehumidity_2m_mean",
    "windspeed_10m_mean", "precipitation_sum",
    "temperature_2m_mean_lag1", "temperature_2m_mean_lag2", "temperature_2m_mean_lag3",
    "relativehumidity_2m_mean_lag1", "relativehumidity_2m_mean_lag2", "relativehumidity_2m_mean_lag3",
    "windspeed_10m_mean_lag1", "windspeed_10m_mean_lag2", "windspeed_10m_mean_lag3",
    "precipitation_sum_lag1", "precipitation_sum_lag2", "precipitation_sum_lag3"
]

# Validaciones
for col in [TARGET_REG, TARGET_CLASS_REAL, FRP_INICIAL_COL]:
    if col not in df.columns:
        raise ValueError(f"❌ Falta columna requerida: {col}")

missing = [c for c in features if c not in df.columns]
if missing:
    raise ValueError(f"❌ Faltan columnas en features: {missing}")


# -----------------------------
# 3) X e y (regresión)
# -----------------------------
# Descripción:
# Se construyen las matrices de entrada (X) y salida (y) para el problema
# de regresión. Se convierten los datos a formato numérico y se eliminan
# observaciones inválidas para garantizar consistencia.

X = df[features].apply(pd.to_numeric, errors="coerce")
y_reg = pd.to_numeric(df[TARGET_REG], errors="coerce")

mask = y_reg.notna() & df[FRP_INICIAL_COL].notna()
X = X.loc[mask].reset_index(drop=True)
y_reg = y_reg.loc[mask].reset_index(drop=True)

y_class_real = df.loc[mask, TARGET_CLASS_REAL].astype(int).reset_index(drop=True)

print("\nDatos para modelar:")
print("X:", X.shape, "| y_reg:", y_reg.shape)


# -----------------------------
# 4) TRAIN / TEST SPLIT
# -----------------------------
# Descripción:
# Se divide el dataset en conjuntos de entrenamiento y prueba, permitiendo
# evaluar el desempeño del modelo sobre datos no vistos.

X_train, X_test, y_train, y_test = train_test_split(
    X, y_reg,
    test_size=0.30,
    random_state=42
)


# -----------------------------
# 5) PIPELINE RIDGE + GRIDSEARCH
# -----------------------------
# Descripción:
# Se construye un pipeline que incluye imputación de valores faltantes,
# escalado de variables y regresión Ridge. Se utiliza GridSearchCV para
# seleccionar el parámetro de regularización óptimo (alpha).

pipe_ridge = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("ridge", Ridge(random_state=42))
])

param_grid = {"ridge__alpha": [0.1, 1, 10, 100, 300]}

gs = GridSearchCV(
    pipe_ridge,
    param_grid,
    scoring="neg_mean_absolute_error",
    cv=5,
    n_jobs=-1,
    verbose=1
)

gs.fit(X_train, y_train)
model = gs.best_estimator_

print("\n✅ Ridge entrenado")
print("Mejor alpha:", gs.best_params_["ridge__alpha"])


# -----------------------------
# 6) MÉTRICAS REGRESIÓN
# -----------------------------
# Descripción:
# Se evalúa el desempeño del modelo de regresión utilizando métricas estándar
# (MAE, RMSE y R²), que permiten cuantificar error y capacidad explicativa.

y_pred_test = model.predict(X_test)

print("\nMétricas regresión (test):")
print("MAE :", mean_absolute_error(y_test, y_pred_test))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))
print("R²  :", r2_score(y_test, y_pred_test))


# -----------------------------
# 7) PREDICCIÓN FRP MAX PARA TODO EL DATASET
# -----------------------------
# Descripción:
# El modelo entrenado se utiliza para predecir el FRP máximo de todos los
# eventos disponibles. Se aplica una restricción física para evitar valores
# negativos.

frp_max_pred = model.predict(X)
frp_max_pred = np.maximum(frp_max_pred, 0)


# -----------------------------
# 8) CLASIFICACIÓN POR FRP ≥ 100
# -----------------------------
# Descripción:
# Se construye una clasificación binaria operativa utilizando un umbral
# fijo de FRP para identificar incendios intensos.

THR_FRP = 100.0
pred_intenso = (frp_max_pred >= THR_FRP).astype(int)


# -----------------------------
# 9) DATASET DE SALIDA CON NUEVAS COLUMNAS
# -----------------------------
# Descripción:
# Se genera un dataset de salida que incorpora las predicciones del modelo
# y la clasificación derivada, manteniendo trazabilidad con los eventos
# originales.

df_out = df.loc[mask].copy().reset_index(drop=True)

df_out["frp_max_pred_ridge"] = frp_max_pred
df_out["pred_intenso_frp100"] = pred_intenso
df_out["umbral_frp"] = THR_FRP

print("\n✅ Dataset con nuevas columnas:")
display(df_out[[
    "event_id_final",
    "frp_inicial",
    "frp_max",
    "frp_max_pred_ridge",
    "pred_intenso_frp100",
    TARGET_CLASS_REAL
]].head())


# -----------------------------
# 10) MÉTRICAS DE CLASIFICACIÓN
# -----------------------------
# Descripción:
# Se evalúa el desempeño del modelo desde una perspectiva de clasificación,
# comparando las predicciones binarias con la etiqueta real de riesgo.

prec = precision_score(y_class_real, pred_intenso, zero_division=0)
rec  = recall_score(y_class_real, pred_intenso, zero_division=0)
f1   = f1_score(y_class_real, pred_intenso, zero_division=0)

print("\n================= MÉTRICAS CLASIFICACIÓN ================")
print(f"Umbral FRP_max_pred ≥ {THR_FRP}")
print(f"Precision: {prec:.3f}")
print(f"Recall   : {rec:.3f}")
print(f"F1       : {f1:.3f}")

print("\nReporte de clasificación:")
print(classification_report(y_class_real, pred_intenso, digits=3, zero_division=0))


# -----------------------------
# 11) MATRIZ DE CONFUSIÓN (PALETA BLUES)
# -----------------------------
# Descripción:
# Se visualiza la matriz de confusión para analizar los aciertos y errores
# del modelo Ridge en la clasificación de incendios intensos.

plot_confusion_with_labels_blues(
    y_class_real,
    pred_intenso,
    title="Matriz de confusión — Ridge (FRP_max_pred ≥ 100)"
)


# -----------------------------
# 12) (OPCIONAL) GUARDAR DATASET
# -----------------------------
# Descripción:
# Se exporta el dataset final con las predicciones del modelo Ridge para
# su uso en análisis posteriores o comparación con otros modelos.

df_out.to_csv(
    "Dataset_Ridge_FRP100_clasificado.csv",
    index=False,
    sep=";",
    decimal="."
)


**Interpretación de las métricas (FRP ≥ 100)**


Al evaluar el desempeño del modelo Ridge Regression bajo el criterio operativo FRP ≥ 100, las métricas relevantes para la clase positiva (evento intenso) son las siguientes:

Precision: 0.371

Recall: 0.038

F1-score: 0.070

Estos valores indican de forma inequívoca que el modelo prácticamente no detecta los eventos de incendio intensos.

**Análisis técnico detallado**

Recall = 0.038 (crítico)

El recall de 0.038 implica que el modelo identifica solo alrededor del 4 % de los eventos intensos reales, perdiendo aproximadamente el 96 % de los incendios con FRP ≥ 100.

En el contexto de un sistema de alerta temprana o priorización operativa, este nivel de sensibilidad es inaceptable, ya que la mayoría de los eventos críticos no serían detectados.

Precision = 0.371

Una precisión de 0.371 indica que, cuando el modelo emite una alerta, acierta en aproximadamente el 37 % de los casos. Este valor no es extremadamente bajo debido a que el modelo emite muy pocas alertas.

Este comportamiento corresponde al patrón típico de un clasificador conservador:

“El modelo casi nunca alerta, pero cuando lo hace, ocasionalmente acierta”.

Accuracy = 0.874 (métrica engañosa)

Aunque la exactitud global alcanza un valor elevado (≈ 0.874), esta métrica no es informativa en este problema.

La razón es el fuerte desbalance de clases:

Clase 0 (no intenso): ~2420 eventos

Clase 1 (intenso): ~339 eventos

El modelo obtiene un accuracy alto simplemente porque predice mayoritariamente la clase 0.
Un accuracy alto no implica un buen modelo, y este punto es crucial de explicitar para evitar interpretaciones erróneas, especialmente en evaluaciones académicas.

Por qué FRP ≥ 100 es especialmente difícil para Ridge

El bajo desempeño del modelo Ridge no es casual, sino que responde a limitaciones estructurales del enfoque lineal en este problema.

1) El umbral se encuentra dentro del rango de error del modelo

Los resultados de regresión muestran:

MAE ≈ 42

RMSE ≈ 76

El umbral operativo de 100 MW se sitúa aproximadamente entre 1 y 2 veces el error típico del modelo.

En este contexto, el modelo carece de resolución suficiente para discriminar de forma confiable si un evento superará o no dicho umbral.

2) La regularización de Ridge atenúa los valores extremos

El valor óptimo de regularización encontrado (α = 300) induce al modelo a penalizar fuertemente coeficientes grandes, lo que se traduce en:

Predicciones más suavizadas,

Subestimación sistemática de valores altos de FRP.

Como consecuencia, los eventos con FRP ≥ 100 quedan mal representados, afectando directamente la capacidad de detección de incendios intensos.

3) El fenómeno subyacente no es lineal

El bajo poder explicativo del modelo (R² ≈ 0.08) evidencia que la relación entre las variables predictoras y el FRP máximo:

presenta interacciones complejas,

incluye comportamientos no lineales,

y responde a dinámicas explosivas y umbrales abruptos.

Un modelo lineal regularizado como Ridge no puede capturar adecuadamente estas características, lo que limita estructuralmente su desempeño.

**Conclusión**

El pobre desempeño del modelo Ridge bajo el criterio FRP ≥ 100 no debe interpretarse como un fallo de implementación, sino como una demostración empírica de sus limitaciones frente a un fenómeno complejo, no lineal y desbalanceado. Este resultado refuerza la necesidad de emplear modelos no lineales, como Random Forest o redes neuronales, para abordar eficazmente este tipo de problemas.